# Feature Transformation with a Amazon SageMaker Processing Job and SparkML

Typically a machine learning (ML) process consists of few steps. First, gathering data with various ETL jobs, then pre-processing the data, featurizing the dataset by incorporating standard techniques or prior knowledge.

Often, distributed data processing frameworks such as Spark are used to pre-process data sets in order to prepare them for training. In this notebook we'll use Amazon SageMaker Processing, and leverage the power of Spark in a managed SageMaker environment to run our processing workload.

![](img/prepare_dataset.png)

![](img/processing.jpg)


## Contents

1. Setup
1. Use Amazon SageMaker Processing Job to execute a Spark ML Transformation
1. Setup Input Data
1. Setup Output Data
1. Build a Spark container for running the processing job
1. Run the Processing Job using Amazon SageMaker
1. Inspect the Processed Output Dta

## Setup

Add the following policies to your SageMaker Execution Role:  
* `EC2ContainerRegistry`
* Permissions: `List`, `Read`, `Write` 
* Repository:  `amazon-reviews-spark-processor`

Let's start by specifying:
* The S3 bucket and prefixes that you use for training and model data. Use the default bucket specified by the Amazon SageMaker session.
* The IAM role ARN used to give processing and training access to the dataset.

In [1]:
import sagemaker
import boto3

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()
region = boto3.Session().region_name

# Using Amazon SageMaker Processing Job to Run a Spark ML Transformation

Get the name of previous Scikit Processing Job to use to construct the input for our Spark ML Processing Job.

# Build a Spark Docker Image to Run the Processing Job

An example Spark container is included in the `./container` directory of this example. The container handles the bootstrapping of all Spark configuration, and serves as a wrapper around the `spark-submit` CLI. At a high level the container provides:
* A set of default Spark/YARN/Hadoop configurations
* A bootstrapping script for configuring and starting up Spark master/worker nodes
* A wrapper around the `spark-submit` CLI to submit a Spark application


After the container build and push process is complete, use the Amazon SageMaker Python SDK to submit a managed, distributed Spark application that performs our dataset processing.

Build the example Spark container.

In [2]:
docker_repo = 'amazon-reviews-spark-processor'
docker_tag = 'latest'

In [3]:
!docker build -t $docker_repo:$docker_tag -f container/Dockerfile ./container

Sending build context to Docker daemon  3.023MB
Step 1/33 : FROM openjdk:8-jre-slim
 ---> bf20b099be53
Step 2/33 : RUN apt-get update
 ---> Using cache
 ---> 568bcfe760c7
Step 3/33 : RUN apt-get install -y curl unzip python3 python3-setuptools python3-pip python-dev python3-dev python-psutil
 ---> Using cache
 ---> 5e8f2be78f61
Step 4/33 : RUN pip3 install py4j psutil==5.6.5 numpy==1.17.4
 ---> Using cache
 ---> f35677fba605
Step 5/33 : RUN apt-get clean
 ---> Using cache
 ---> c3d1f7a6fe79
Step 6/33 : RUN rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> 251601397074
Step 7/33 : ENV PYTHONHASHSEED 0
 ---> Using cache
 ---> e409300c31d9
Step 8/33 : ENV PYTHONIOENCODING UTF-8
 ---> Using cache
 ---> 9124e9a57eb0
Step 9/33 : ENV PIP_DISABLE_PIP_VERSION_CHECK 1
 ---> Using cache
 ---> c7d3e56eeae5
Step 10/33 : ENV HADOOP_VERSION 3.0.0
 ---> Using cache
 ---> 2f920737f94b
Step 11/33 : ENV HADOOP_HOME /usr/hadoop-$HADOOP_VERSION
 ---> Using cache
 ---> 06620a5da59c
Step 12/33 : ENV HADOOP

Create an Amazon Elastic Container Registry (Amazon ECR) repository for the Spark container and push the image.

In [4]:
import boto3
account_id = boto3.client('sts').get_caller_identity().get('Account')
region = boto3.session.Session().region_name

image_uri = '{}.dkr.ecr.{}.amazonaws.com/{}:{}'.format(account_id, region, docker_repo, docker_tag)
print(image_uri)

086401037028.dkr.ecr.us-west-2.amazonaws.com/amazon-reviews-spark-processor:latest


Create ECR repository and push docker image

In [5]:
!$(aws ecr get-login --region $region --registry-ids $account_id --no-include-email)

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [6]:
!aws ecr describe-repositories --repository-names $docker_repo || aws ecr create-repository --repository-name $docker_repo


An error occurred (RepositoryNotFoundException) when calling the DescribeRepositories operation: The repository with name 'amazon-reviews-spark-processor' does not exist in the registry with id '086401037028'
{
    "repository": {
        "repositoryArn": "arn:aws:ecr:us-west-2:086401037028:repository/amazon-reviews-spark-processor",
        "registryId": "086401037028",
        "repositoryName": "amazon-reviews-spark-processor",
        "repositoryUri": "086401037028.dkr.ecr.us-west-2.amazonaws.com/amazon-reviews-spark-processor",
        "createdAt": 1587838682.0,
        "imageTagMutability": "MUTABLE",
        "imageScanningConfiguration": {
            "scanOnPush": false
        }
    }
}


In [7]:
!docker tag $docker_repo:$docker_tag $image_uri

In [8]:
!docker push $image_uri

The push refers to repository [086401037028.dkr.ecr.us-west-2.amazonaws.com/amazon-reviews-spark-processor]

64ea2c17: Preparing 
0ee2b131: Preparing 
ba328002: Preparing 
1eb866e9: Preparing 
6f8023d3: Preparing 
fa33a696: Preparing 
6b9d6e4d: Preparing 
7cc5f91d: Preparing 
eb2ae899: Preparing 
3a42c7f7: Preparing 
a9b45c60: Preparing 
3b6fbb91: Preparing 
ac5e9bd9: Preparing 
0d58a380: Preparing 
3a42c7f7: Pushed   490.3MB/481.8MBPushing  245.7MB/481.8MBlatest: digest: sha256:121bfffc5888bf5f38ee5d331e5aada2bad3a12231cc30ca62c6ab0d9c3f87be size: 3472


# Run the Job using Amazon SageMaker Processing Jobs

Next, use the Amazon SageMaker Python SDK to submit a processing job. Use the Spark container that was just built, and a SparkML script for processing in the job configuration.

Review the Spark processing script.

In [9]:
cat preprocess-spark-text-to-tfidf.py

from __future__ import print_function
from __future__ import unicode_literals

import time
import sys
import os
import shutil
import csv

import pyspark
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, DateType
from pyspark.sql.functions import *
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml.linalg import DenseVector
from pyspark.sql.functions import split
from pyspark.sql.functions import udf, col
from pyspark.sql.types import ArrayType, DoubleType
from pyspark.ml.feature import PCA, StandardScaler


def to_array(col):
    def to_array_internal(v):
        if v:
            return v.toArray().tolist()
        else:
            print('EmptyV: {}'.format(v))
            return []
    return udf(to_array_internal, ArrayType(DoubleType())).asNondeterministic()(col)


def transform(spark, s3_input_data, s3_output_train_data, s3_output_validation_data, s3_output_tes

In [10]:
from sagemaker.processing import ScriptProcessor

processor = ScriptProcessor(base_job_name='spark-amazon-reviews-processor',
                            image_uri=image_uri,
                            command=['/opt/program/submit'],
                            role=role,
                            instance_count=2, # instance_count needs to be > 1 or you will see the following error:  "INFO yarn.Client: Application report for application_ (state: ACCEPTED)"
                            instance_type='ml.r5.8xlarge',
                            env={'mode': 'python'})

# Start the Spark Processing Job

_Notes on Invoking from Lambda:_
* However, if we use the boto3 SDK (ie. with a Lambda), we need to copy the `preprocess-text-to-tfidf.py` file to S3 and specify the everything include --py-files, etc.
* We would need to do the following before invoking the Lambda:
     !aws s3 cp preprocess.py s3://<location>/sagemaker/spark-preprocess-reviews-demo/code/preprocess.py
     !aws s3 cp preprocess.py s3://<location>/sagemaker/spark-preprocess-reviews-demo/py_files/preprocess.py
* Then reference the s3://<location> above in the --py-files, etc.
* See Lambda example code in this same project for more details.

_Notes on not using ProcessingInput and Output:_
* Since Spark natively reads/writes from/to S3 using s3a://, we can avoid the copy required by ProcessingInput and ProcessingOutput (FullyReplicated or ShardedByS3Key) and just specify the S3 input and output buckets/prefixes._"
* See https://github.com/awslabs/amazon-sagemaker-examples/issues/994 for issues related to using /opt/ml/processing/input/ and output/
* If we use ProcessingInput, the data will be copied to each node (which we don't want in this case since Spark already handles this)

# Setup Input Data

In [11]:
# Inputs
s3_input_data = 's3://{}/amazon-reviews-pds/tsv/'.format(bucket)
print(s3_input_data)

s3://sagemaker-us-west-2-086401037028/amazon-reviews-pds/tsv/


In [12]:
!aws s3 ls $s3_input_data

2020-04-25 16:31:58   18997559 amazon_reviews_us_Digital_Software_v1_00.tsv.gz
2020-04-25 16:32:07   27442648 amazon_reviews_us_Digital_Video_Games_v1_00.tsv.gz


# Setup Output Data

In [13]:
from time import gmtime, strftime
timestamp_prefix = strftime("%Y-%m-%d-%H-%M-%S", gmtime())

output_prefix = 'amazon-reviews-spark-processor-{}'.format(timestamp_prefix)

In [14]:
train_data_tfidf_output = 's3://{}/{}/output/tfidf-train'.format(bucket, output_prefix)
validation_data_tfidf_output = 's3://{}/{}/output/tfidf-validation'.format(bucket, output_prefix)
test_data_tfidf_output = 's3://{}/{}/output/tfidf-test'.format(bucket, output_prefix)

print(train_data_tfidf_output)
print(validation_data_tfidf_output)
print(test_data_tfidf_output)

s3://sagemaker-us-west-2-086401037028/amazon-reviews-spark-processor-2020-04-25-18-18-29/output/tfidf-train
s3://sagemaker-us-west-2-086401037028/amazon-reviews-spark-processor-2020-04-25-18-18-29/output/tfidf-validation
s3://sagemaker-us-west-2-086401037028/amazon-reviews-spark-processor-2020-04-25-18-18-29/output/tfidf-test


In [15]:
from sagemaker.processing import ProcessingOutput

processor.run(code='preprocess-spark-text-to-tfidf.py',
              arguments=['s3_input_data', s3_input_data,
                         's3_output_train_data', train_data_tfidf_output,
                         's3_output_validation_data', validation_data_tfidf_output,
                         's3_output_test_data', test_data_tfidf_output,                         
              ],
              # We need this dummy output to allow us to call 
              #    ProcessingJob.from_processing_name() later 
              #    to describe the job and poll for Completed status
              outputs=[
                       ProcessingOutput(s3_upload_mode='EndOfJob',
                                        output_name='dummy-output',
                                        source='/opt/ml/processing/output')
              ],          
              logs=True,
              wait=False
)


Job Name:  spark-amazon-reviews-processor-2020-04-25-18-18-29-763
Inputs:  [{'InputName': 'code', 'S3Input': {'S3Uri': 's3://sagemaker-us-west-2-086401037028/spark-amazon-reviews-processor-2020-04-25-18-18-29-763/input/code/preprocess-spark-text-to-tfidf.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'dummy-output', 'S3Output': {'S3Uri': 's3://sagemaker-us-west-2-086401037028/spark-amazon-reviews-processor-2020-04-25-18-18-29-763/output/dummy-output', 'LocalPath': '/opt/ml/processing/output', 'S3UploadMode': 'EndOfJob'}}]


In [16]:
from IPython.core.display import display, HTML

spark_processing_job_name = processor.jobs[-1].describe()['ProcessingJobName']

display(HTML('<b>Review <a href="https://console.aws.amazon.com/cloudwatch/home?region={}#logStream:group=/aws/sagemaker/ProcessingJobs;prefix={};streamFilter=typeLogStreamPrefix">CloudWatch Logs</a> After About 5 Minutes</b>'.format(region, spark_processing_job_name)))


In [17]:
from IPython.core.display import display, HTML

# This is different than the job name because we are not using ProcessingOutput's in this Spark ML case.
spark_processing_job_s3_output_prefix = output_prefix

display(HTML('<b>Review <a href="https://s3.console.aws.amazon.com/s3/buckets/{}/{}/?region={}&tab=overview">S3 Output Data</a> After The Spark Job Has Completed</b>'.format(bucket, spark_processing_job_s3_output_prefix, region)))


# Please Wait Until the Processing Job Completes
Re-run this next cell until the job status shows `Completed`.

In [18]:
running_processor = sagemaker.processing.ProcessingJob.from_processing_name(processing_job_name=spark_processing_job_name,
                                                                            sagemaker_session=sagemaker_session)

processing_job_description = running_processor.describe()

processing_job_status = processing_job_description['ProcessingJobStatus']
print('\n')
print(processing_job_status)
print('\n')

print(processing_job_description)



InProgress


{'ProcessingInputs': [{'InputName': 'code', 'S3Input': {'S3Uri': 's3://sagemaker-us-west-2-086401037028/spark-amazon-reviews-processor-2020-04-25-18-18-29-763/input/code/preprocess-spark-text-to-tfidf.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}], 'ProcessingOutputConfig': {'Outputs': [{'OutputName': 'dummy-output', 'S3Output': {'S3Uri': 's3://sagemaker-us-west-2-086401037028/spark-amazon-reviews-processor-2020-04-25-18-18-29-763/output/dummy-output', 'LocalPath': '/opt/ml/processing/output', 'S3UploadMode': 'EndOfJob'}}]}, 'ProcessingJobName': 'spark-amazon-reviews-processor-2020-04-25-18-18-29-763', 'ProcessingResources': {'ClusterConfig': {'InstanceCount': 2, 'InstanceType': 'ml.r5.8xlarge', 'VolumeSizeInGB': 30}}, 'StoppingCondition': {'MaxRuntimeInSeconds': 86400}, 'AppSpecification': {'ImageUri': '086401037028.dkr.ecr.us-west-2.amazonaws

# Inspect the Processed Output Dataset


## The next cells will not work properly until the job completes above.


Take a look at a few rows of the transformed dataset to make sure the processing was successful.

In [19]:
!aws s3 ls --recursive $train_data_tfidf_output/

In [20]:
!aws s3 ls --recursive $validation_data_tfidf_output/

In [21]:
!aws s3 ls --recursive $test_data_tfidf_output/